# Import

In [1]:
import re
import pandas        as pd
import numpy         as np

from datetime        import datetime

# Load DataSet

In [2]:
df = pd.read_csv('../dataset/df_final.csv', encoding='utf-8', sep=';')


In [3]:
df.head()

,Unnamed: 0,id_ads,manufacturer_by,model,version,km,year,price,seller,address,scrapy_datetime,zip_code,city
0,0,8c9aef9c-1513-41ab-8db4-10686decdfb5,bmw,116,d 5p Msport auto,146000,2019,16500,Bryan Auto Srl,BRYAN AUTO • IT-13100 Vercelli,2024-11-22,13100,Vercelli
1,1,78b02ce1-2c8f-4caf-88fc-7d59b6e42cda,bmw,x1,xdrive18d Business auto my18,100000,2018,18999,Bryan Auto Srl,BRYAN AUTO • IT-13100 Vercelli,2024-11-22,13100,Vercelli
2,2,a564e061-226c-464f-bf1a-b178e7b929dc,bmw,116,116d 5p. Msport,53050,2019,19900,Bryan Auto Srl,BRYAN AUTO • IT-13100 Vercelli,2024-11-22,13100,Vercelli
3,3,6e6ae0b6-d199-4d2e-96ec-d525bfb48e1e,bmw,730,d xDrive Eccelsa,226135,2018,25900,Bryan Auto Srl,BRYAN AUTO • IT-13100 Vercelli,2024-11-22,13100,Vercelli
4,4,888c30ce-7225-4e02-a41e-f30970df33c1,bmw,x1,xdrive25e Msport auto,119500,2020,21500,Bryan Auto Srl,BRYAN AUTO • IT-13100 Vercelli,2024-11-22,13100,Vercelli


## 01 - Clean columns

In [4]:
df_01 = df.copy()

In [5]:
df_01 = df_01.drop('Unnamed: 0', axis=1)
#df = df.drop('coluna_indesejada', axis=1)


In [6]:
df_01.head(1)

,id_ads,manufacturer_by,model,version,km,year,price,seller,address,scrapy_datetime,zip_code,city
0,8c9aef9c-1513-41ab-8db4-10686decdfb5,bmw,116,d 5p Msport auto,146000,2019,16500,Bryan Auto Srl,BRYAN AUTO • IT-13100 Vercelli,2024-11-22,13100,Vercelli


## Creating Non-Sale History Column

In [7]:
df_02 = df_01.copy()

In [8]:
# Contar valores únicos e mapear
count_id_ads = df_02['id_ads'].value_counts()
df_02['count_id'] = df_02['id_ads'].map(count_id_ads)

In [9]:
df_02.shape

(398, 13)

In [10]:
qntd = len(df_02['id_ads'].unique())
print(qntd)

380


## 03 - Creating Today's Date Column

In [11]:
df_03 = df_02.copy()

In [12]:
df_03['today'] = pd.to_datetime(datetime.now().strftime('%Y/%m/%d'))



In [13]:
# COnvertendo os tipos das colunas
df_03['scrapy_datetime'] = pd.to_datetime(df_03['scrapy_datetime']).dt.date
df_03['scrapy_datetime'] = pd.to_datetime(df_03['scrapy_datetime'])
# Use pd.to_datetime() para converter a coluna "DataHora" em objetos datetime
#df['DataHora'] = pd.to_datetime(df['DataHora']).dt.date
#
#df_03['scrapy_datetime'] = 
## Use dt.date para extrair apenas a data
#df['Data'] = df['DataHora'].dt.date


In [14]:
df_03.dtypes


id_ads                     object
manufacturer_by            object
model                      object
version                    object
km                          int64
year                        int64
price                       int64
seller                     object
address                    object
scrapy_datetime    datetime64[ns]
zip_code                    int64
city                       object
count_id                    int64
today              datetime64[ns]
dtype: object

In [15]:
df_03.head(1)

,id_ads,manufacturer_by,model,version,km,year,price,seller,address,scrapy_datetime,zip_code,city,count_id,today
0,8c9aef9c-1513-41ab-8db4-10686decdfb5,bmw,116,d 5p Msport auto,146000,2019,16500,Bryan Auto Srl,BRYAN AUTO • IT-13100 Vercelli,2024-11-22,13100,Vercelli,1,2024-11-22


### Adding the number of days the vehicle has been on sale

In [16]:
# Selecione apenas as linhas onde Coluna1 >= 2 e Coluna2 ocorre mais de uma vez
condicao = (df_03['count_id'] >= 2) & df_03['id_ads'].duplicated(keep=False)

# Filtrar as linhas que atendem à condição
df_4 = df_03[condicao]

# Calcular a diferença em dias
df_4['Dias_Passados'] = abs(df_4['scrapy_datetime'] - df_4['today'])



C:\Users\bruno\AppData\Local\Temp\ipykernel_7900\1912781929.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_4['Dias_Passados'] = abs(df_4['scrapy_datetime'] - df_4['today'])


In [17]:
df_4.head()

,id_ads,manufacturer_by,model,version,km,year,price,seller,address,scrapy_datetime,zip_code,city,count_id,today,Dias_Passados
2,a564e061-226c-464f-bf1a-b178e7b929dc,bmw,116,116d 5p. Msport,53050,2019,19900,Bryan Auto Srl,BRYAN AUTO • IT-13100 Vercelli,2024-11-22,13100,Vercelli,2,2024-11-22,0 days
3,6e6ae0b6-d199-4d2e-96ec-d525bfb48e1e,bmw,730,d xDrive Eccelsa,226135,2018,25900,Bryan Auto Srl,BRYAN AUTO • IT-13100 Vercelli,2024-11-22,13100,Vercelli,2,2024-11-22,0 days
4,888c30ce-7225-4e02-a41e-f30970df33c1,bmw,x1,xdrive25e Msport auto,119500,2020,21500,Bryan Auto Srl,BRYAN AUTO • IT-13100 Vercelli,2024-11-22,13100,Vercelli,2,2024-11-22,0 days
5,ba7cd9b3-4e1f-4373-8975-5120395e92d8,bmw,318,F30 2015 Berlina Diese 318d Luxury auto,26260,2018,20900,Bryan Auto Srl,BRYAN AUTO • IT-13100 Vercelli,2024-11-22,13100,Vercelli,2,2024-11-22,0 days
6,250bff01-d3f5-4e26-b9cb-743bede208ee,bmw,325,d xDrive Touring Msport,217024,2018,11890,Bryan Auto Srl,BRYAN AUTO • IT-13100 Vercelli,2024-11-22,13100,Vercelli,2,2024-11-22,0 days


## DataFrame Final

In [18]:
df_final = pd.DataFrame(df_4)

In [19]:
df_final.head()

,id_ads,manufacturer_by,model,version,km,year,price,seller,address,scrapy_datetime,zip_code,city,count_id,today,Dias_Passados
2,a564e061-226c-464f-bf1a-b178e7b929dc,bmw,116,116d 5p. Msport,53050,2019,19900,Bryan Auto Srl,BRYAN AUTO • IT-13100 Vercelli,2024-11-22,13100,Vercelli,2,2024-11-22,0 days
3,6e6ae0b6-d199-4d2e-96ec-d525bfb48e1e,bmw,730,d xDrive Eccelsa,226135,2018,25900,Bryan Auto Srl,BRYAN AUTO • IT-13100 Vercelli,2024-11-22,13100,Vercelli,2,2024-11-22,0 days
4,888c30ce-7225-4e02-a41e-f30970df33c1,bmw,x1,xdrive25e Msport auto,119500,2020,21500,Bryan Auto Srl,BRYAN AUTO • IT-13100 Vercelli,2024-11-22,13100,Vercelli,2,2024-11-22,0 days
5,ba7cd9b3-4e1f-4373-8975-5120395e92d8,bmw,318,F30 2015 Berlina Diese 318d Luxury auto,26260,2018,20900,Bryan Auto Srl,BRYAN AUTO • IT-13100 Vercelli,2024-11-22,13100,Vercelli,2,2024-11-22,0 days
6,250bff01-d3f5-4e26-b9cb-743bede208ee,bmw,325,d xDrive Touring Msport,217024,2018,11890,Bryan Auto Srl,BRYAN AUTO • IT-13100 Vercelli,2024-11-22,13100,Vercelli,2,2024-11-22,0 days


## Save the final dataset


In [21]:
#rows = df_final.shape[0]

df_final.to_csv('../dataset/df_final_model.csv', encoding='utf-8', sep=';')

In [22]:
df_final.shape

(36, 15)